In [1]:
import pandas as pd

In [2]:
m1ohm2 = '2023-09-15 AFO BASE_Macro1_CruceOnHand_Macro2.xlsb'

## Parte I: Resumen x OSO

In [3]:
df = pd.read_excel(m1ohm2, sheet_name = 'Resumen x OSO', engine = 'pyxlsb', dtype = 'string', 
                   skiprows = 2, skipfooter = 1, usecols = 'A:K')

In [4]:
df['Orden Superior'].ffill(inplace = True)
df['Orden'].ffill(inplace = True)

In [5]:
Orden_Superior = []
for i in range(len(df['Orden Superior'])):
    Orden_Superior.append(df['Orden Superior'][i][:10] + '00')

In [6]:
df.pop('Orden Superior')
df.insert(loc = 0, column = 'Orden Superior', value = Orden_Superior)

In [7]:
df['OSO'] = df['Orden'] + df['Operación']
oso = df.pop('OSO')
df.insert(loc = 3, column ='OSO', value = oso.values)

In [8]:
df.insert(loc = 4, column ='St.Consolidacion', value = pd.NA)

In [9]:
df['Orden Superior'] = df['Orden Superior'].astype('string')
df['St.Consolidacion'] = df['St.Consolidacion'].astype('string')
for i in df.columns.values[-2:]:
    df[i] = df[i].astype('float')

In [10]:
df['Cuenta de OH 8034'].fillna(0, inplace = True)
df['Cuenta de OH Otros'].fillna(0, inplace = True)

In [11]:
df['Cuenta de Material'] = df['Cuenta de OH 8034'] + df['Cuenta de OH Otros']

In [12]:
df.loc[df['Cuenta de OH 8034'] == 0, 'Cuenta de OH 8034'] = pd.NA
df.loc[df['Cuenta de OH Otros'] == 0, 'Cuenta de OH Otros'] = pd.NA

In [13]:
for i in range(len(df['Orden Superior'])):
    if df['Cuenta de Material'][i] > 0 and pd.isna(df['Cuenta de OH Otros'][i]) == True and df['ST OSO'][i] == 'Completa':
        df.loc[[i],['St.Consolidacion']] = 'Consolidada'
    elif df['Cuenta de Material'][i] > 0 and pd.isna(df['Cuenta de OH Otros'][i]) == False and df['ST OSO'][i] == 'Completa':
        df.loc[[i],['St.Consolidacion']] = 'No Consolidada'
        continue

In [14]:
# Correción errores detectados el 02-07-2022:
df['LnPend x OSO'] = df['LnPend x OSO'].astype('int')
df['LnPend x Hija'] = df['LnPend x Hija'].astype('int')
df['LnPend x Padre'] = df['LnPend x Padre'].astype('int')

# Eliminamos correlativo
df.set_index('Orden Superior', inplace = True)

## Parte II: Resumen x Linea

In [15]:
df2 = pd.read_excel(m1ohm2, sheet_name = 'Resumen x Ln', engine = 'pyxlsb', dtype = 'string', usecols = 'A:U')

In [16]:
# Ajustamos el formato de algunas columnas:
for i in df2.columns[6:14]:
    df2[i] = df2[i].astype('float')
df2['LnPend x OSO'] = df2['LnPend x OSO'].astype('float')
df2['LnPend x Hija'] = df2['LnPend x Hija'].astype('float') 
df2['LnPend x Padre'] = df2['LnPend x Padre'].astype('float') 

In [17]:
for i in range(len(df2['ST Linea'])):
    if df2['ST Linea'][i] == 'Completa' and df2['Delta'][i] > 0:
        df2.loc[[i],['ST Linea']] = 'Completa - Sobre Consumo'
        continue

In [18]:
df2.set_index('OS+OP', inplace = True)

In [19]:
with pd.ExcelWriter('AM1-OHA_AM2.xlsx') as writer:  
    df.to_excel(writer, sheet_name='Resumen x OSO')
    df2.to_excel(writer, sheet_name='Resumen x Ln')